In [1]:
import pandas as pd
from pennylane import numpy as np
import math
import pennylane as qml
import random
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import itertools

2024-03-15 15:43:04.434855: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tf.random.set_seed(15)
random.seed(42)
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams.update({'font.size':20})
legend_prop = {'weight':'bold'}
from pylab import rcParams
rcParams['axes.linewidth'] = 2

# Data

In [3]:
df = pd.read_csv('Processed_data.csv')
df = df.drop([ 'Unnamed: 0'], axis=1)
# Used data from Aug 2013 till 1st March 2024 
df=df[500:]
# Remove few redundant data
df=df.drop(columns=['Date'])
df=df.drop(columns=['CUMLOGRET_1'])
df=df.drop(columns=['Gold in USD volume'])
df=df.drop(columns=['Open'])
df=df.drop(columns=['High'])
df=df.drop(columns=['Low'])

In [4]:
close_min=min(df['Close'].values)
close_max=max(df['Close'].values)
a=-1
b=1
normalized_df_2=(b-a)*((df-df.min())/(df.max()-df.min()))+a

In [5]:
a=0
b=np.pi
normalized_df=(b-a)*((df-df.min())/(df.max()-df.min()))+a

In [6]:
normalized_df['Close']=normalized_df_2['Close']

In [7]:
df=normalized_df.copy()

In [8]:
df['add1']=0

In [9]:
# Set target and features
target = "Close"
features = list(df.columns.difference(["Close"]))
print(features)
print(len(features))

['3M', 'BBB_20_2.0', 'BBL_20_2.0', 'BBM_20_2.0', 'BBP_20_2.0', 'BBU_20_2.0', 'Close_copy', 'Crude Futures_close', 'Crude Futures_volume', 'Crude_H-L', 'Crude_O-C', 'EMA_14', 'EMA_21', 'EMA_7', 'FTSE_H-L', 'FTSE_O-C', 'GBP USD ', 'GBP_USD_H-L', 'GBP_USD_O-C', 'Gold in USD close', 'Gold_H-L', 'Gold_O-C', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'SMA_14', 'SMA_21', 'SMA_7', 'Volume', 'add1']
30


In [10]:
df.head()

,Close,Volume,EMA_7,EMA_14,EMA_21,SMA_7,SMA_14,SMA_21,MACD_12_26_9,MACDh_12_26_9,...,FTSE_H-L,FTSE_O-C,GBP_USD_H-L,GBP_USD_O-C,Gold_H-L,Gold_O-C,Crude_H-L,Crude_O-C,Close_copy,add1
500,0.117799,0.810869,1.570742,1.352452,1.214416,1.591382,1.498519,1.405322,2.772839,1.910741,...,0.293720,1.126378,0.238316,1.190780,0.920019,1.243063,0.127631,1.040208,1.755835,0
501,0.095219,0.545773,1.581258,1.364587,1.227323,1.595772,1.503892,1.418889,2.774454,1.906973,...,0.283627,1.399911,0.448918,1.102716,0.819899,1.454706,0.166851,1.051275,1.720366,0
502,0.076480,0.498673,1.581000,1.370115,1.235421,1.600817,1.509387,1.433529,2.762821,1.875403,...,0.395664,1.383211,0.249414,1.355422,0.719779,1.666348,0.206071,1.062341,1.690931,0
503,0.066283,0.577589,1.576375,1.372192,1.240805,1.608703,1.512196,1.442771,2.745473,1.837829,...,0.258898,1.346068,0.126425,1.086763,0.389655,1.248421,0.111677,1.023901,1.674913,0
504,0.004701,0.529924,1.546141,1.357600,1.233754,1.600864,1.501484,1.442658,2.696586,1.739792,...,0.491047,1.569501,0.671479,1.132071,0.787428,0.834067,0.154221,1.026230,1.578181,0


In [11]:
sequence_length = 30
window =6

def create_dataset(dataset,target,features, lookback, window):
    X_store, y_store = [], []
    for i in range(0,len(dataset)-lookback-window,3):
        X = (dataset[features].values)[i:i+lookback]
        y = (dataset[target].values)[i+lookback:i+lookback+window]
        X_store.append(X)
        y_store.append(y)
    return np.array(X_store), np.array(y_store)

In [12]:
# Split train test data
from sklearn.model_selection import train_test_split
# df_train, df_test = train_test_split(df, test_size=0.05)
size=int(len(df)*0.96)
df_train=df[:size]
df_test=df[size:]
print('Total sample', len(df))
print('Train sample', len(df_train))
print('Test sample', len(df_test))

Total sample 2758
Train sample 2647
Test sample 111


In [13]:
# Train data
X_train, y_train = create_dataset(
    df_train,
    target=target,
    features=features,
    lookback=sequence_length, window=window)

indices = tf.range(start=0, limit=tf.shape(X_train)[0])
shuffled_indices = tf.random.shuffle(indices)
X_train = tf.gather(X_train, shuffled_indices)
y_train = tf.gather(y_train, shuffled_indices)

print(X_train.shape, y_train.shape)

(871, 30, 30) (871, 6)


2024-03-15 15:43:06.179598: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [14]:
# Test data
X_test, y_test = create_dataset(
    df_test,
    target=target,
    features=features,
    lookback=sequence_length, window=window)

print(X_test.shape, y_test.shape)

(25, 30, 30) (25, 6)


In [15]:
X_train[0][0].numpy()

array([0.49807091, 0.40893943, 1.95671783, 1.78139437, 1.09359525,
       1.64147411, 1.75999501, 0.90040677, 0.68958889, 0.56636414,
       1.35704688, 1.63383859, 1.62012948, 1.71959071, 0.22054233,
       1.40307772, 0.93827426, 0.22520515, 1.20354332, 0.65706315,
       0.22188682, 1.18412503, 2.14917488, 1.76476801, 2.14425442,
       1.715655  , 1.78350133, 1.75220262, 0.1543872 , 0.        ])

# QNN

In [16]:
nqubits = 10
device = qml.device('default.qubit', wires=nqubits)
# Define Quantum layer(1)
@qml.qnode(device)
def quantum_circuit_1(inputs, params):
    
    m=3
    k1=0 ; k2=k1+m ; k3=k2+m ; k4=k3+m ; k5=k4+m ; k6=k5+m ; k7=k6+m ; k8=k7+m ; k9=k8+m ; k10=k9+m ; k11=k10+m
        
    for p1,p2,p3,p4,p5,p6,p7,p8,p9,p10 in zip(params[k1:k2],params[k2:k3],params[k3:k4],
                                                  params[k4:k5],params[k5:k6],params[k6:k7],params[k7:k8],
                                                  params[k8:k9],params[k9:k10],params[k10:k11]):
        
        qml.Rot(*inputs[:3], wires=0)
        qml.Rot(*inputs[3:6], wires=1)
        qml.Rot(*inputs[6:9], wires=2)
        qml.Rot(*inputs[9:12], wires=3)
        qml.Rot(*inputs[12:15], wires=4)
        qml.Rot(*inputs[15:18], wires=5)
        qml.Rot(*inputs[18:21], wires=6)
        qml.Rot(*inputs[21:24], wires=7)
        qml.Rot(*inputs[24:27], wires=8)
        qml.Rot(*inputs[27:30], wires=9)
        
        qml.Rot(*p1, wires=0)
        qml.Rot(*p2, wires=1)
        qml.Rot(*p3, wires=2)
        qml.Rot(*p4, wires=3)
        qml.Rot(*p5, wires=4)
        qml.Rot(*p6, wires=5)
        qml.Rot(*p7, wires=6)
        qml.Rot(*p8, wires=7)
        qml.Rot(*p9, wires=8)
        qml.Rot(*p10, wires=9)
        
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[3, 4]) 
        qml.CNOT(wires=[4, 5])
        qml.CNOT(wires=[5, 6]) 
        qml.CNOT(wires=[6, 7])
        qml.CNOT(wires=[7, 8])
        qml.CNOT(wires=[8, 9])
        
    return [qml.expval(qml.PauliZ(k)) for k in range(nqubits)]

In [17]:
device = qml.device('default.qubit', wires=12)
# Define Quantum layer(2)
@qml.qnode(device)
def quantum_circuit_2(inputs, params):
    
    qml.RX(inputs[0], wires=0)
    qml.RX(inputs[1], wires=1)
    qml.RX(inputs[2], wires=2)
    qml.RX(inputs[3], wires=3)
    qml.RX(inputs[4], wires=4)
    qml.RX(inputs[5], wires=5)
    qml.RX(inputs[6], wires=6)
    qml.RX(inputs[7], wires=7)
    qml.RX(inputs[8], wires=8)
    qml.RX(inputs[9], wires=9)
    qml.RX(inputs[10], wires=10)
    qml.RX(inputs[11], wires=11)
    
    m=2
    k1=1234 ; k2=k1+m ; k3=k2+m ; k4=k3+m ; k5=k4+m ; k6=k5+m ; k7=k6+m ; k8=k7+m ; k9=k8+m ; k10=k9+m ; k11=k10+m
    k12=k11+m ; k13=k12+m 
    
#     print('Parameters in Second Qauntum circuit is till',k13)
#     print('Total parameter in Circuit 2',k13-k1)
        
    for p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12 in zip(params[k1:k2],params[k2:k3],params[k3:k4],
                                                  params[k4:k5],params[k5:k6],params[k6:k7],params[k7:k8],
                                                  params[k8:k9],params[k9:k10],params[k10:k11],params[k11:k12]
                                                      ,params[k12:k13]):
        
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[3, 4]) 
        qml.CNOT(wires=[4, 5])
        qml.CNOT(wires=[5, 6]) 
        qml.CNOT(wires=[6, 7])
        qml.CNOT(wires=[7, 8])
        qml.CNOT(wires=[8, 9])
        qml.CNOT(wires=[9, 10])
        qml.CNOT(wires=[10, 11])
        
        qml.Rot(*p1, wires=0)
        qml.Rot(*p2, wires=1)
        qml.Rot(*p3, wires=2)
        qml.Rot(*p4, wires=3)
        qml.Rot(*p5, wires=4)
        qml.Rot(*p6, wires=5)
        qml.Rot(*p7, wires=6)
        qml.Rot(*p8, wires=7)
        qml.Rot(*p9, wires=8)
        qml.Rot(*p10, wires=9)
        qml.Rot(*p10, wires=10)
        qml.Rot(*p11, wires=11)
        
    return [qml.expval(qml.PauliZ(k)) for k in range(12)]

In [38]:
# Define loss function
def loss_func(params,X,y):
    
    cost=0
    
    for j in range(len(X)):
        
        v1=[quantum_circuit_1(X[j][i].numpy(), params) for i in range(X.shape[1])]  

        # Flatten 
        flat_list = list(itertools.chain(*v1))
        v1_=np.array(flat_list)
    #     print('v1 shape is',v1_.shape)    
    #     print('Total parameter in Circuit 1',60)

        m1=300
        l1=30 ; l2=l1+m1 ; l3=l2+m1  ; l4=l3+m1 ; l5=l4+m1 
        l6=l5+1 ; l7=l6+1 ; l8=l7+1 
    #     print('Parameters in NN layer 1 start at',l1)

        v2= np.add(np.concatenate((params[l5],params[l6],params[l7],params[l8])),np.dot(np.transpose(
                                                                    v1_),
                                                                                       np.hstack((params[l1:l2],
                                                                                                  params[l2:l3],params[l3:l4],params[l4:l5])) ))


        v3=np.where(v2 > 0, v2, v2 * 0.05) 
    #     print('Parameters in NN layer 1 end at',l8)
    #     print('Total parameter in NN 1',l8-l1+1)

        l9=l8+1
    #     print('Parameters in Next circuit start at',l9)
        v4=quantum_circuit_2(np.array(v3),params)

        l11=1258
        m1=12
        l12=l11 ; l13=l12+m1 ; l14=l13+m1 ; l15=l14+m1 ; l16=l15+m1 ; l17=l16+m1
        l18=l17+1 ; l19=l18+1 ; l20=l19+1 ; l21=l20+1 
    #     print('Parameters in last layers start at',l12)
        v5= np.add(np.concatenate((params[l17],params[l18],params[l19],params[l20],params[l21])),np.dot(np.transpose(
                                                                    v4),
                                                                                       np.hstack((params[l12:l13],
                                                                                                  params[l13:l14],params[l14:l15],params[l15:l16],params[l16:l17])) ))
        v6=np.where(v5 > 0, v5, v5 * 0.05)

        m1=15
        l23=l21+1 ; l24=l23+m1 ; l25=l24+m1 
        l27=l25 ; l28=l27+1 
        v7 = np.add(np.concatenate((params[l27],params[l28])),np.dot(np.transpose(
                                                                    v6),
                                                                                       np.hstack((params[l23:l24],
                                                                                       params[l24:l25])) ))
    #     print('Last parameters is',l28)
    #     print('Total number of parameters in last is',l28-l12+1)

        cost=cost+np.mean((y[j] - v7) ** 2)

    return cost/len(X)

In [19]:
# Define cost function
def cost_fn(params,**kwargs):
    z=kwargs['batch']
    
#     print('Batch is',z)
    
    v1=[quantum_circuit_1(X_train[z][i].numpy(), params) for i in range(X_train.shape[1])]    
    
    # Flatten 
    flat_list = list(itertools.chain(*v1))
    v1_=np.array(flat_list)
#     print('v1 shape is',v1_.shape)    
#     print('Total parameter in Circuit 1',60)
    
    m1=300
    l1=30 ; l2=l1+m1 ; l3=l2+m1  ; l4=l3+m1 ; l5=l4+m1 
    l6=l5+1 ; l7=l6+1 ; l8=l7+1 
#     print('Parameters in NN layer 1 start at',l1)
    
    v2= np.add(np.concatenate((params[l5],params[l6],params[l7],params[l8])),np.dot(np.transpose(
                                                                v1_),
                                                                                   np.hstack((params[l1:l2],
                                                                                              params[l2:l3],params[l3:l4],params[l4:l5])) ))

    
    v3=np.where(v2 > 0, v2, v2 * 0.05) 
#     print('Parameters in NN layer 1 end at',l8)
#     print('Total parameter in NN 1',l8-l1+1)
    
    l9=l8+1
#     print('Parameters in Next circuit start at',l9)
    v4=quantum_circuit_2(np.array(v3),params)
    
    l11=1258
    m1=12
    l12=l11 ; l13=l12+m1 ; l14=l13+m1 ; l15=l14+m1 ; l16=l15+m1 ; l17=l16+m1
    l18=l17+1 ; l19=l18+1 ; l20=l19+1 ; l21=l20+1 
#     print('Parameters in last layers start at',l12)
    v5= np.add(np.concatenate((params[l17],params[l18],params[l19],params[l20],params[l21])),np.dot(np.transpose(
                                                                v4),
                                                                                   np.hstack((params[l12:l13],
                                                                                              params[l13:l14],params[l14:l15],params[l15:l16],params[l16:l17])) ))
    v6=np.where(v5 > 0, v5, v5 * 0.05)
    
    m1=15
    l23=l21+1 ; l24=l23+m1 ; l25=l24+m1 
    l27=l25 ; l28=l27+1 
    v7 = np.add(np.concatenate((params[l27],params[l28])),np.dot(np.transpose(
                                                                v6),
                                                                                   np.hstack((params[l23:l24],
                                                                                   params[l24:l25])) ))
#     print('Last parameters is',l28)
#     print('Total number of parameters in last is',l28-l12+1)

    cost=np.mean((y_train[z] - v7) ** 2)
    return cost

In [20]:
params_1=np.random.uniform(0,np.pi, size=(30, 3),requires_grad = True)
params_2=np.random.normal(-0.05,0.05, size=(1204, 3),requires_grad = True)
params_3=np.random.normal(0,np.pi, size=(24, 3),requires_grad = True)
params_4=np.random.normal(-0.05,0.05, size=(97, 3),requires_grad = True)

params=np.vstack([params_1,params_2,params_3,params_4])

In [21]:
opt = qml.AdamOptimizer(stepsize=0.005)

In [22]:
len(params)

1355

In [23]:
# batch_size=32 # Batch size
batch_list=list(range(0,X_train.shape[0]))

In [24]:
batch_=list(range(len(batch_list)))

In [25]:
store_loss_test=[]
store_val=[]
store_params=[]

In [26]:
import time

In [ ]:
for i in range (15):
    print('Epoch',i,'running')
    batch_=list(range(len(batch_list)))
    while len(batch_)>0:
        j= random.choice(batch_)
        # start=time.time()
        params, prev_cost = opt.step_and_cost(cost_fn,params,batch=j)
        # end=time.time()
        # print(start-end)
        batch_.remove(j)
    store_val.append(i)
    store_params.append(params)
    print('-------------------------------------------')

Epoch 0 running
-------------------------------------------
Epoch 1 running
-------------------------------------------
Epoch 2 running


In [ ]:
opt.stepsize

In [ ]:
# Make pred for check data
def pred_(X,params):
    v1=[quantum_circuit_1(X[i].numpy(), params) for i in range(X.shape[1])]    
    
    # Flatten 
    flat_list = list(itertools.chain(*v1))
    v1_=np.array(flat_list)
#     print('v1 shape is',v1_.shape)    
#     print('Total parameter in Circuit 1',100)
    
    m1=300
    l1=30 ; l2=l1+m1 ; l3=l2+m1  ; l4=l3+m1 ; l5=l4+m1 
    l6=l5+1 ; l7=l6+1 ; l8=l7+1 
#     print('Parameters in NN layer 1 start at',l1)
    
    v2= np.add(np.concatenate((params[l5],params[l6],params[l7],params[l8])),np.dot(np.transpose(
                                                                v1_),
                                                                                   np.hstack((params[l1:l2],
                                                                                              params[l2:l3],params[l3:l4],params[l4:l5])) ))

    
    v3=np.where(v2 > 0, v2, v2 * 0.05) 
#     print('Parameters in NN layer 1 end at',l8)
#     print('Total parameter in NN 1',l8-l1+1)
    
    l9=l8+1
#     print('Parameters in Next circuit start at',l9)
    v4=quantum_circuit_2(np.array(v3),params)
    
    l11=1258
    m1=12
    l12=l11 ; l13=l12+m1 ; l14=l13+m1 ; l15=l14+m1 ; l16=l15+m1 ; l17=l16+m1
    l18=l17+1 ; l19=l18+1 ; l20=l19+1 ; l21=l20+1 
#     print('Parameters in last layers start at',l12)
    v5= np.add(np.concatenate((params[l17],params[l18],params[l19],params[l20],params[l21])),np.dot(np.transpose(
                                                                v4),
                                                                                   np.hstack((params[l12:l13],
                                                                                              params[l13:l14],params[l14:l15],params[l15:l16],params[l16:l17])) ))
    v6=np.where(v5 > 0, v5, v5 * 0.05)
    
    m1=15
    l23=l21+1 ; l24=l23+m1 ; l25=l24+m1 
    l27=l25 ; l28=l27+1 
    v7 = np.add(np.concatenate((params[l27],params[l28])),np.dot(np.transpose(
                                                                v6),
                                                                                   np.hstack((params[l23:l24],
                                                                                   params[l24:l25])) ))
    return v7

In [ ]:
def mae_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        sum_error += abs(predicted[i] - actual[i])
    return sum_error/len(actual)

def rmse_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        sum_error += (predicted[i] - actual[i])**2
    return (sum_error/len(actual))**0.5

In [106]:
store_test_loss=[]

In [107]:
for i in store_params:
    store_test_loss.append(loss_func(i,X_test,y_test))
    print(store_test_loss)

[tensor(0.02449016, requires_grad=True)]
[tensor(0.02449016, requires_grad=True), tensor(0.0064217, requires_grad=True)]
[tensor(0.02449016, requires_grad=True), tensor(0.0064217, requires_grad=True), tensor(0.00509536, requires_grad=True)]
[tensor(0.02449016, requires_grad=True), tensor(0.0064217, requires_grad=True), tensor(0.00509536, requires_grad=True), tensor(0.0057044, requires_grad=True)]
[tensor(0.02449016, requires_grad=True), tensor(0.0064217, requires_grad=True), tensor(0.00509536, requires_grad=True), tensor(0.0057044, requires_grad=True), tensor(0.01290248, requires_grad=True)]
[tensor(0.02449016, requires_grad=True), tensor(0.0064217, requires_grad=True), tensor(0.00509536, requires_grad=True), tensor(0.0057044, requires_grad=True), tensor(0.01290248, requires_grad=True), tensor(0.00993529, requires_grad=True)]
[tensor(0.02449016, requires_grad=True), tensor(0.0064217, requires_grad=True), tensor(0.00509536, requires_grad=True), tensor(0.0057044, requires_grad=True), ten

In [108]:
store_test_loss

[tensor(0.02449016, requires_grad=True),
 tensor(0.0064217, requires_grad=True),
 tensor(0.00509536, requires_grad=True),
 tensor(0.0057044, requires_grad=True),
 tensor(0.01290248, requires_grad=True),
 tensor(0.00993529, requires_grad=True),
 tensor(0.00577412, requires_grad=True),
 tensor(0.00334069, requires_grad=True),
 tensor(0.00886539, requires_grad=True),
 tensor(0.01837415, requires_grad=True),
 tensor(0.00923507, requires_grad=True),
 tensor(0.01734177, requires_grad=True),
 tensor(0.00726066, requires_grad=True),
 tensor(0.0082363, requires_grad=True),
 tensor(0.00663217, requires_grad=True)]

In [109]:
store_train_loss=[]

In [ ]:
for i in store_params:
    store_train_loss.append(loss_func(i,X_train,y_train))
    print(store_train_loss)

[0.023613082609789244]
[0.023613082609789244, 0.01854311117123644]
[0.023613082609789244, 0.01854311117123644, 0.012453520764812055]
[0.023613082609789244, 0.01854311117123644, 0.012453520764812055, 0.011403825514160703]
[0.023613082609789244, 0.01854311117123644, 0.012453520764812055, 0.011403825514160703, 0.013556711651261882]


In [114]:
store_train_loss

[0.023613082609789244,
 0.01854311117123644,
 0.012453520764812055,
 0.011403825514160703,
 0.013556711651261882,
 0.021247254999760927,
 0.009597506202448482,
 0.011372007761889036,
 0.008032843816189104,
 0.012890795674827217,
 0.012634871133502552,
 0.009603613692778139,
 0.0095107931024976,
 0.007118442013375939,
 0.0072810770337642374]

In [96]:
b=1 ; a=-1

In [97]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
def cal_metrics(y_true,y_pred):
    mae=0
    rmse=0
    for i in range(len(y_true)):
        y1=((((y_true[i]-a)/(b-a))*(close_max - close_min))+close_min)
        y2=((((y_pred[i]-a)/(b-a))*(close_max - close_min))+close_min)
        mae=mean_absolute_error(y1, y2)+mae
        rmse=mean_squared_error(y1, y2, squared=False)+rmse  
    return mae/len(y_true) , rmse/len(y_true)

In [98]:
y_test_loss=[]

for i in range(len(y_test)):
    y_test_loss.append(pred_(X_test[i],store_params[-1]))

In [99]:
import warnings
warnings.filterwarnings('ignore')

In [100]:
mae_test_pros, rmse_test_pros= cal_metrics(y_test,y_test_loss)
print('MAE test error is',mae_test_pros)
print('RMSE test error is',rmse_test_pros)

MAE test error is 104.39677636314363
RMSE test error is 110.72698457359417


In [102]:
y_train_loss=[]

for i in range(len(y_train)):
    y_train_loss.append(pred_(X_train[i],store_params[-1]))

In [103]:
mae_train_pros, rmse_train_pros= cal_metrics(y_train,y_train_loss)
print('MAE train error is',mae_train_pros)
print('RMSE train error is',rmse_train_pros)

MAE train error is 97.63707057105405
RMSE train error is 107.87771965385195
